<a href="https://colab.research.google.com/github/phamthien300820/CS114.K21/blob/master/ML_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from tqdm import tqdm
!pip install mahotas
import mahotas
from sklearn.preprocessing import LabelEncoder 
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import h5py
from sklearn.ensemble import RandomForestClassifier


In [ ]:
#Đường dẫn output
output_path = "/content/drive/My Drive/X_i"

# Đường dẫn dữ liệu training
train_path = "/content/drive/My Drive/Machine_Learning Helmet"

# get the training labels
train_labels = os.listdir(train_path)
train_labels.sort()

# fixed-sizes for image
fixed_size = tuple((100, 100))

# bins for histogram
bins = 8

# empty lists to hold feature vectors and labels
global_features = []
labels = []

# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

In [ ]:
train_labels

['Có người đi xe máy', 'Không có xe máy']

In [ ]:
for training_name in train_labels:
    
    dir = os.path.join(train_path, training_name)
    # nhãn đang được train hiện tại
    current_label = training_name
    # Duyệt tất cả các ảnh trong mỗi tập con
    for image in tqdm(os.listdir(dir)):
        
        try:
          path = os.path.join(dir,image)
          #Đọc ảnh và resize lại theo fixed_size
          image = cv2.imread(path)
          image = cv2.resize(image, fixed_size)

          ####################################
          # Global Feature extraction
          ####################################
          fv_hu_moments = fd_hu_moments(image)
          fv_haralick   = fd_haralick(image)
          fv_histogram  = fd_histogram(image)

          ###################################
          # Concatenate global features
          ###################################
          global_feature = np.hstack([fv_histogram, fv_hu_moments, fv_haralick])

          # update the list of labels and feature vectors
          labels.append(current_label)
          global_features.append(global_feature)
        except:
            pass

    print("[STATUS] processed folder: {}".format(current_label))

print("[STATUS] completed Global Feature Extraction...")


# get the overall feature vector size
print ("[STATUS] feature vector size {}".format(np.array(global_features).shape))

# get the overall training label size
print ("[STATUS] training Labels {}".format(np.array(labels).shape))

# encode the target labels
le = LabelEncoder()
target = le.fit_transform(labels)

# normalize the feature vector in the range (0-1)
scaler = MinMaxScaler(feature_range=(0, 1))
rescaled_features = scaler.fit_transform(global_features)

# save the feature vector using HDF5
h5f_data = h5py.File(output_path+'data.h5', 'w')
h5f_data.create_dataset('dataset_1', data=np.array(rescaled_features))

h5f_label = h5py.File(output_path+'labels.h5', 'w')
h5f_label.create_dataset('dataset_1', data=np.array(target))

h5f_data.close()
h5f_label.close()

print("[STATUS] end of training..")




  0%|          | 0/834 [00:00<?, ?it/s]


  0%|          | 1/834 [00:00<04:54,  2.83it/s]


  0%|          | 2/834 [00:00<05:04,  2.73it/s]


  0%|          | 3/834 [00:01<05:13,  2.65it/s]


  0%|          | 4/834 [00:01<04:59,  2.77it/s]


  1%|          | 5/834 [00:01<04:38,  2.97it/s]


  1%|          | 6/834 [00:02<04:16,  3.23it/s]


  1%|          | 7/834 [00:02<03:59,  3.45it/s]


  1%|          | 8/834 [00:02<03:49,  3.61it/s]


  1%|          | 9/834 [00:02<03:43,  3.69it/s]


  1%|          | 10/834 [00:02<03:36,  3.80it/s]


  1%|▏         | 11/834 [00:03<03:32,  3.88it/s]


  1%|▏         | 12/834 [00:03<03:31,  3.89it/s]


  2%|▏         | 13/834 [00:03<03:31,  3.89it/s]


  2%|▏         | 14/834 [00:04<03:28,  3.93it/s]


  2%|▏         | 15/834 [00:04<03:27,  3.95it/s]


  2%|▏         | 16/834 [00:04<03:28,  3.91it/s]


  2%|▏         | 17/834 [00:04<03:28,  3.91it/s]


  2%|▏         | 18/834 [00:05<03:27,  3.92it/s]


  2%|▏         | 19/834 [00:05<03:28,  3.92it/

[STATUS] processed folder: Có người đi xe máy





  1%|          | 1/151 [00:00<00:31,  4.69it/s]


  1%|▏         | 2/151 [00:00<00:36,  4.08it/s]


  2%|▏         | 3/151 [00:00<00:39,  3.71it/s]


  3%|▎         | 4/151 [00:01<00:42,  3.48it/s]


  3%|▎         | 5/151 [00:01<00:43,  3.34it/s]


  4%|▍         | 6/151 [00:01<00:40,  3.62it/s]


  5%|▍         | 7/151 [00:01<00:37,  3.86it/s]


  5%|▌         | 8/151 [00:02<00:40,  3.56it/s]


  6%|▌         | 9/151 [00:02<00:42,  3.38it/s]


  7%|▋         | 10/151 [00:02<00:42,  3.28it/s]


  7%|▋         | 11/151 [00:03<00:43,  3.20it/s]


  8%|▊         | 12/151 [00:03<00:40,  3.41it/s]


  9%|▉         | 14/151 [00:03<00:30,  4.48it/s]


 11%|█▏        | 17/151 [00:03<00:22,  5.90it/s]


 13%|█▎        | 20/151 [00:03<00:17,  7.69it/s]


 15%|█▌        | 23/151 [00:04<00:13,  9.73it/s]


 17%|█▋        | 26/151 [00:04<00:10, 11.91it/s]


 19%|█▉        | 29/151 [00:04<00:08, 14.17it/s]


 21%|██        | 32/151 [00:04<00:07, 15.74it/s]


 23%|██▎       | 35/151 [00:04<00:06,

[STATUS] processed folder: Không có xe máy
[STATUS] completed Global Feature Extraction...
[STATUS] feature vector size (921, 532)
[STATUS] training Labels (921,)
[STATUS] end of training..


In [ ]:
output_path = "\content\X"

# fixed-sizes for image
#fixed_size = tuple((100, 100))

# Số lượng cây ở RandomForest
num_trees = 300

# bins for histogram
bins = 8

# số hình ảnh mỗi lớp
images_per_class = 10;

# nhập vector đặc trưng và nhãn được training
h5f_data = h5py.File('/content/drive/My Drive/X_idata.h5', 'r')
h5f_label = h5py.File('/content/drive/My Drive/X_ilabels.h5', 'r')

global_features_string = h5f_data['dataset_1']
global_labels_string = h5f_label['dataset_1']

global_features = np.array(global_features_string)
global_labels = np.array(global_labels_string)

h5f_data.close()
h5f_label.close()


# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

# feature-descriptor-2: Haralick Texture
def fd_haralick(image):
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

# feature-descriptor-3: Color Histogram
def fd_histogram(image, mask=None):
    # Chuyển đổi hình ảnh sang không gian màu HSV
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # Tính toán biểu đồ màu
    hist  = cv2.calcHist([image], [0, 1, 2], None, [bins, bins, bins], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    
    return hist.flatten()


#  Tạo model - Random Forests
clf  = RandomForestClassifier(n_estimators=num_trees)
clf.fit(global_features, global_labels)

# Đường dẫn đến dữ liệu test
test_path = "/content/drive/My Drive/Machine_Learning Helmet"
# lấy labels của test
test_labels = os.listdir(test_path)

# sắp xếp labels của test
test_labels.sort()
print(test_labels)
# loop through the test images
test_features = []
test_results = []
for testing_name in test_labels:
    # join the training data path and each species training folder
    dir = os.path.join(test_path, testing_name)
    # get the current training label
    current_label = testing_name
    # loop over the images in each sub-folder
    for image in tqdm(os.listdir(dir)):
        # get the image file name
        try:
          path = os.path.join(dir,image)
          # read the image and resize it oto a fixed-size
          image = cv2.imread(path)
          image = cv2.resize(image, fixed_size)

          ####################################
          # Global Feature extraction
          ####################################
          fv_hu_moments = fd_hu_moments(image)
          fv_haralick   = fd_haralick(image)
          fv_histogram  = fd_histogram(image)

          ###################################
          # Concatenate global features
          ###################################

          # update the list of labels and feature vectors
          test_results.append(current_label)
          test_features.append(np.hstack([fv_histogram, fv_hu_moments, fv_haralick]))
        except:
            pass

# predict label of test image
le = LabelEncoder()
y_result = le.fit_transform(test_results)
y_pred = clf.predict(test_features)

print(y_pred)
print("Result: ", (y_pred == y_result).tolist().count(True)/len(y_result))




  0%|          | 0/834 [00:00<?, ?it/s]

['Có người đi xe máy', 'Không có xe máy']





  0%|          | 1/834 [00:00<03:02,  4.56it/s]


  0%|          | 2/834 [00:00<03:08,  4.41it/s]


  0%|          | 3/834 [00:00<03:14,  4.28it/s]


  0%|          | 4/834 [00:00<03:16,  4.22it/s]


  1%|          | 5/834 [00:01<03:20,  4.14it/s]


  1%|          | 6/834 [00:01<03:21,  4.10it/s]


  1%|          | 7/834 [00:01<03:23,  4.06it/s]


  1%|          | 8/834 [00:01<03:23,  4.05it/s]


  1%|          | 9/834 [00:02<03:24,  4.03it/s]


  1%|          | 10/834 [00:02<03:24,  4.03it/s]


  1%|▏         | 11/834 [00:02<03:25,  4.01it/s]


  1%|▏         | 12/834 [00:02<03:26,  3.99it/s]


  2%|▏         | 13/834 [00:03<03:26,  3.98it/s]


  2%|▏         | 14/834 [00:03<03:26,  3.97it/s]


  2%|▏         | 15/834 [00:03<03:27,  3.94it/s]


  2%|▏         | 16/834 [00:03<03:25,  3.97it/s]


  2%|▏         | 17/834 [00:04<03:24,  3.99it/s]


  2%|▏         | 18/834 [00:04<03:27,  3.94it/s]


  2%|▏         | 19/834 [00:04<03:30,  3.88it/s]


  2%|▏         | 20/834 [00:05<03:28,

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 

In [95]:
from sklearn.metrics import classification_report
print(classification_report(y_result, y_pred, labels=[0, 1]))

              precision    recall  f1-score   support

           0       0.96      0.95      0.95       770
           1       0.75      0.78      0.76       151

    accuracy                           0.92       921
   macro avg       0.85      0.86      0.86       921
weighted avg       0.92      0.92      0.92       921



In [ ]:
import os
import cv2
import numpy as np
from numpy import linalg as LA

IMG = '/content/drive/My Drive/Machine_Learning Helmet/Có người đi xe máy/008BD75395323E9D7EB38C2A2ED623B4.jpg'

def hog(img_gray, cell_size=8, block_size=2, bins=9):
    img = img_gray
    h, w = img.shape # 128, 64
    
    # gradient
    xkernel = np.array([[-1, 0, 1]])
    ykernel = np.array([[-1], [0], [1]])
    dx = cv2.filter2D(img, cv2.CV_32F, xkernel)
    dy = cv2.filter2D(img, cv2.CV_32F, ykernel)
    
    # histogram
    magnitude = np.sqrt(np.square(dx) + np.square(dy))
    orientation = np.arctan(np.divide(dy, dx+0.00001)) # radian
    orientation = np.degrees(orientation) # -90 -> 90
    orientation += 90 # 0 -> 180
    
    num_cell_x = w // cell_size # 8
    num_cell_y = h // cell_size # 16
    hist_tensor = np.zeros([num_cell_y, num_cell_x, bins]) # 16 x 8 x 9
    for cx in range(num_cell_x):
        for cy in range(num_cell_y):
            ori = orientation[cy*cell_size:cy*cell_size+cell_size, cx*cell_size:cx*cell_size+cell_size]
            mag = magnitude[cy*cell_size:cy*cell_size+cell_size, cx*cell_size:cx*cell_size+cell_size]
            # https://docs.scipy.org/doc/numpy/reference/generated/numpy.histogram.html
            hist, _ = np.histogram(ori, bins=bins, range=(0, 180), weights=mag) # 1-D vector, 9 elements
            hist_tensor[cy, cx, :] = hist
        pass
    pass
    
    # normalization
    redundant_cell = block_size-1
    feature_tensor = np.zeros([num_cell_y-redundant_cell, num_cell_x-redundant_cell, block_size*block_size*bins])
    for bx in range(num_cell_x-redundant_cell): # 7
        for by in range(num_cell_y-redundant_cell): # 15
            by_from = by
            by_to = by+block_size
            bx_from = bx
            bx_to = bx+block_size
            v = hist_tensor[by_from:by_to, bx_from:bx_to, :].flatten() # to 1-D array (vector)
            feature_tensor[by, bx, :] = v / LA.norm(v, 2)
            # avoid NaN:
            if np.isnan(feature_tensor[by, bx, :]).any(): # avoid NaN (zero division)
                feature_tensor[by, bx, :] = v
    
    return feature_tensor.flatten() # 3780 features

def main(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(src=img, dsize=(64, 128))
    f = hog(img)
    print('Extracted feature vector of %s. Shape:' % img_path)
    print('Feature size:', f.shape)
    print('Features (HOG):', f)
    pass

if __name__ == "__main__":
    print('Start running HOG on image @ %s' % IMG)
    main(IMG)

Start running HOG on image @ /content/drive/My Drive/Machine_Learning Helmet/Có người đi xe máy/008BD75395323E9D7EB38C2A2ED623B4.jpg
Extracted feature vector of /content/drive/My Drive/Machine_Learning Helmet/Có người đi xe máy/053142803.jpg. Shape:
Feature size: (3780,)
Features (HOG): [0.1662788  0.13844891 0.07729993 ... 0.18587949 0.00082554 0.02222006]
